In [1]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

#import src
import Cleanup
import Transformation
import FileProcessing

In [2]:
#define the objects to be called later. 
cleanup=Cleanup.Cleanup()
transform=Transformation.Transformation()
fileProcessing=FileProcessing.FileProcessing()

Read the File from Data folder and remove all duplicates. 

In [3]:
file_name='Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data'
ngsim=fileProcessing.read_input(file_name)

original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Data File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data


Remove any Duplicate records from the original File. 

In [4]:
cleanup.remove_dups(ngsim)

704000 duplicate values have been removed


,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,...,D_Zone,Int_ID,Section_ID,Direction,Movement,Preceding,Following,Space_Headway,Time_Headway,Location
0,515,2330,1123,1118848075000,30.034,188.062,6451203.729,1873252.549,13.0,6.9,...,NaN,NaN,NaN,NaN,NaN,500,523,119.10,5.11,us-101
2,2224,6548,1902,1113437421700,41.429,472.901,6042814.264,2133542.012,14.3,6.9,...,NaN,NaN,NaN,NaN,NaN,2208,2211,53.34,2.01,i-80
3,2127,6459,567,1118847624800,19.632,1775.614,6452425.122,1872172.475,13.5,6.9,...,NaN,NaN,NaN,NaN,NaN,2124,2132,48.92,1.30,us-101
4,1033,4827,592,1118848324700,6.202,1701.144,6452347.673,1872258.452,13.5,4.4,...,NaN,NaN,NaN,NaN,NaN,1029,1040,38.81,0.92,us-101
6,1890,9157,628,1118849672700,53.514,817.521,6451655.238,1872800.663,24.0,8.5,...,NaN,NaN,NaN,NaN,NaN,1882,1897,102.65,2.27,us-101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11850521,1355,5622,938,1113438127100,18.275,1464.336,6042648.149,2134520.896,12.3,6.8,...,NaN,NaN,NaN,NaN,NaN,1351,1361,26.91,15.92,i-80
11850522,1474,5688,401,1113438133700,5.509,560.412,6042767.840,2133624.549,17.8,7.9,...,NaN,NaN,NaN,NaN,NaN,1468,1480,74.49,1.73,i-80
11850523,398,2368,654,1113437801700,18.903,635.841,6042771.741,2133701.076,15.2,8.5,...,NaN,NaN,NaN,NaN,NaN,385,406,90.95,3.77,i-80
11850524,599,875,577,1113436854400,77.094,621.358,6042831.282,2133693.854,15.3,6.4,...,NaN,NaN,NaN,NaN,NaN,0,611,0.00,0.00,i-80


Feet to Metres:

Below Four parametrs consider distance in Feet and they are converted to metre. Speed to Metre/ second from Feet/second and acceleration from feet/second Square to Metre/Second Square. 

Drop the columns we wont use in the model.
Add Relative time which is the time from the first timframe in the dataset. This is calculated by using the minimum value ofGlobal time(absolute time from 1970)

In [5]:
ngsim=transform.convert_feet_to_metre(ngsim)

In [6]:
ngsim.shape

(11146526, 25)

In [7]:
ngsim=transform.drop_not_required_columns(ngsim)

Create Placeholders for the user defined columns we will be populating later, to be used in the model. 
Map the Vehicle classes to their english equivalent. 
Filter the original file into US-101 and I-80 to perform and populate the required fields later. These files contain same values of Vehicle ID used for multiple vehicles based on the highway. 



In [8]:
#filtered_ngsim= ngsim[((ngsim['Location'] == 'us-101') | (ngsim['Location'] == 'i-80')) & (ngsim['Preceding']>0) & (ngsim['Following']>0) ]
ngsim=transform.create_column_placeholders(ngsim)

Split data into US-101 and I-80 specific dataframes.

In [9]:
filtered_ngsim_U,filtered_ngsim_I = transform.bifurcate_highways(ngsim)

Create Sets to use and create the L-F Pairs. 

In [10]:
v_Class_M_U,v_Class_C_U,v_Class_HV_U = transform.class_vehicle_sets(filtered_ngsim_U)
v_Class_M_I,v_Class_C_I,v_Class_HV_I = transform.class_vehicle_sets(filtered_ngsim_I)


Add the Preceding Vehicle Class to the new previous class column

In [11]:
filtered_ngsim_U=transform.preceding_vehicle_class(filtered_ngsim_U,v_Class_M_U,v_Class_C_U,v_Class_HV_U)
filtered_ngsim_I=transform.preceding_vehicle_class(filtered_ngsim_I,v_Class_M_I,v_Class_C_I,v_Class_HV_I)

Map the Vehicle Length of the previous vehicle into the row. 

In [12]:
filtered_ngsim_U=transform.preceding_vehicle_length(filtered_ngsim_U)
filtered_ngsim_I=transform.preceding_vehicle_length(filtered_ngsim_I)

Convert the Front to Front Space Headway to Back Bumper (preceding Vehicle) to front Bumper(of Following ) Headway. This will give the actual distance between the vehicle ignoring the vehicle length.
Find the time it would take for the Following vehicle to reach the rear Bumper of the Preceding Vehicle. 

In [13]:
filtered_ngsim_U= transform.front_to_front_bumper_details_changed_to_rear_to_front_bumper_details(filtered_ngsim_U)
filtered_ngsim_I= transform.front_to_front_bumper_details_changed_to_rear_to_front_bumper_details(filtered_ngsim_I)

In [14]:
filtered_ngsim_U.columns

Index(['Vehicle_ID', 'Frame_ID', 'Global_Time', 'Local_X', 'Local_Y',
       'v_length', 'v_Class', 'v_Vel', 'v_Acc', 'Lane_ID', 'Preceding',
       'Following', 'Space_Headway', 'Time_Headway', 'Location',
       'Preceding_Vehicle_Class', 'Rear_to_Front_Space_Headway',
       'Front_To_Rear_Time_Headway', 'Velocity Difference_Following-Preceding',
       'Acceleration Difference_Following-Preceding', 'L-F_Pair',
       'v_Class_Name', 'Vehicle_combination', 'preceding_vehicle_length'],
      dtype='object')

Vehicle Combination of the Preceding and the Following Vehicle. 

In [15]:
filtered_ngsim_U=transform.map_preceding_vehicle_details(filtered_ngsim_U)
filtered_ngsim_I=transform.map_preceding_vehicle_details(filtered_ngsim_I)

us-101::621 cars dont change lanes
us-101::2226 cars Change lanes
i-80::817 cars dont change lanes
i-80::2184 cars Change lanes


In [16]:
filtered_ngsim_U=transform.map_pairs(filtered_ngsim_U)
filtered_ngsim_I=transform.map_pairs(filtered_ngsim_I)

#filtered_ngsim_U=cleanup.filter_records_for_model(filtered_ngsim_U)
#filtered_ngsim_I=cleanup.filter_records_for_model(filtered_ngsim_I)



In [17]:
df=filtered_ngsim_I
loc = df['Location'][0]
v_Class_verify = df[['Vehicle_ID', 'v_Class']]
v_Class_verify.drop_duplicates(inplace=True)
v_Class_verify = v_Class_verify.groupby(
    ['Vehicle_ID'], as_index=False).count()
remove_bad_data_vehicles = set(
    v_Class_verify[v_Class_verify["v_Class"] > 1]['Vehicle_ID'])
bad_v_Class_length_curr = df[(
    df['Vehicle_ID'].isin(remove_bad_data_vehicles))]
bad_v_Class_length_prev = df[(
    df['Preceding'].isin(remove_bad_data_vehicles))]
bad_v_Class_length = pd.concat(
    [bad_v_Class_length_curr, bad_v_Class_length_prev])
lf_pair_remove_first_last_5_seconds_lane1, lf_pair_remove_first_last_5_seconds_lane2, lf_pair_remove_first_last_5_seconds_lane3 = cleanup.lane_change_info(
    df)

In [18]:
bad_v_Class_length

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration
0,36,4,1113433135300,8.268005,23.032822,7.55904,3,3.377184,0.0,3,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.0,88.3
1,36,5,1113433135400,8.268005,23.337926,7.55904,3,3.377184,0.0,3,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.1,88.3
2,36,6,1113433135500,8.268005,23.642422,7.55904,3,3.377184,0.0,3,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.2,88.3
3,36,7,1113433135600,8.268310,23.794822,7.55904,3,3.377184,0.0,3,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.3,88.3
4,36,8,1113433135700,8.420100,24.709222,7.55904,3,3.377184,0.0,3,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.4,88.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4566024,2460,11502,1113438715100,16.737482,478.060207,4.66344,2,3.319272,0.0,5,...,7.02564,True,2445,1.030224,0.624840,True,499.102380,3.0,4.7,5.1
4566026,2460,11503,1113438715200,16.741750,478.392134,4.66344,2,3.319272,0.0,5,...,7.02564,True,2445,1.136904,1.420368,True,499.203574,3.0,4.8,5.1
4566030,2460,11504,1113438715300,16.746322,478.724062,4.66344,2,3.319272,0.0,5,...,7.02564,True,2445,1.136904,0.000000,True,499.311778,3.0,4.9,5.1
4566034,2460,11505,1113438715400,16.750589,479.055684,4.66344,2,3.319272,0.0,5,...,7.02564,True,2445,1.136904,0.000000,True,499.464482,3.0,5.0,5.1


In [19]:
filtered_ngsim_I

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration
0,36,4,1113433135300,8.268005,23.032822,7.55904,3,3.377184,0.000000,3,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,0.0,88.3
1,36,5,1113433135400,8.268005,23.337926,7.55904,3,3.377184,0.000000,3,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,0.1,88.3
2,36,6,1113433135500,8.268005,23.642422,7.55904,3,3.377184,0.000000,3,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,0.2,88.3
3,36,7,1113433135600,8.268310,23.794822,7.55904,3,3.377184,0.000000,3,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,0.3,88.3
4,36,8,1113433135700,8.420100,24.709222,7.55904,3,3.377184,0.000000,3,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,0.4,88.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4566382,2485,11687,1113438733600,16.769182,498.415361,4.20624,2,3.557016,-1.094232,5,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,5.8,6.2
4566383,2485,11688,1113438733700,16.771315,498.778073,4.20624,2,3.307080,-3.413760,5,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,5.9,6.2
4566384,2485,11689,1113438733800,16.774363,499.163340,4.20624,2,3.307080,0.000000,5,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,6.0,6.2
4566385,2485,11690,1113438733900,16.775278,499.315740,4.20624,2,3.307080,0.000000,5,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,6.1,6.2


In [20]:

both_lane_change_1 = df[(df['L-F_Pair'].isin(lf_pair_remove_first_last_5_seconds_lane1)) & (df['Lane_ID'] == 1) &
                        ((df['pair_Time_Duration'] < 5) | (df['pair_Time_Duration'] > (df['total_pair_duration'] - 5)))]
both_lane_change_2 = df[(df['L-F_Pair'].isin(lf_pair_remove_first_last_5_seconds_lane2)) & (df['Lane_ID'] == 2) &
                        ((df['pair_Time_Duration'] < 5) | (df['pair_Time_Duration'] > (df['total_pair_duration'] - 5)))]
both_lane_change_3 = df[(df['L-F_Pair'].isin(lf_pair_remove_first_last_5_seconds_lane3)) & (df['Lane_ID'] == 3) &
                        ((df['pair_Time_Duration'] < 5) | (df['pair_Time_Duration'] > (df['total_pair_duration'] - 5)))]


In [21]:
both_lane_change_1

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration
3680,44,292,1113433164100,2.169262,19.368516,4.05384,2,8.049768,0.000000,1,...,6.33984,True,3355,1.755648,-0.009144,True,68.142917,2.0,0.0,18.8
3736,44,293,1113433164200,2.171090,20.130516,4.05384,2,8.049768,0.000000,1,...,6.33984,True,3355,1.752600,0.000000,True,68.318482,2.0,0.1,18.8
3743,44,294,1113433164300,2.171700,21.050402,4.05384,2,8.049768,0.000000,1,...,6.33984,True,3355,1.755648,0.000000,True,68.494046,2.0,0.2,18.8
3800,44,295,1113433164400,2.171700,21.964498,4.05384,2,8.049768,0.000000,1,...,6.33984,True,3355,1.755648,-0.012192,True,68.669306,2.0,0.3,18.8
3827,44,296,1113433164500,2.171395,22.726498,4.05384,2,8.049768,0.000000,1,...,6.33984,True,3355,1.752600,0.009144,True,68.844871,2.0,0.4,18.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4541360,2487,10538,1113438618700,1.504188,495.143333,4.05384,2,11.832336,3.166872,1,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,4.2,4.6
4541415,2487,10539,1113438618800,1.512418,496.357961,4.05384,2,11.993880,0.771144,1,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,4.3,4.6
4541457,2487,10540,1113438618900,1.520952,497.590877,4.05384,2,11.993880,0.000000,1,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,4.4,4.6
4541526,2487,10541,1113438619000,1.529182,498.810077,4.05384,2,11.993880,0.000000,1,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,4.5,4.6


In [22]:

time_headway_less_than5 = df[(df['Time_Headway'] > 5)]
remove_ramp_data = df[(df['Lane_ID'] >= 4)]


total_duration_less_than_minute = df[(
    df['total_pair_duration'] < 30)]

before = df.shape[0]
print(f"dataset before Row Removal{df.shape}")

print(
    f"{loc}: {both_lane_change_1.shape[0]} vehicles first and Last 5 seconds removed from Lane 1 due to lane changing")
print(
    f"{loc}: {both_lane_change_2.shape[0]} vehicles first and Last 5 seconds removed from Lane 2 due to lane changing")
print(
    f"{loc}: {both_lane_change_3.shape[0]} vehicles first and Last 5 seconds removed from Lane 3 due to lane changing")

print(
    f"{loc}: {bad_v_Class_length.shape[0]} have multiple lengths and classes for same Vehicle ID")
print(
    f"{loc}: {total_duration_less_than_minute.shape[0]} have total car following less than 30 seconds")
print(
    f"{loc}: {time_headway_less_than5.shape[0]} have time Headway less than 5 seconds")
print(
    f"{loc}: {remove_ramp_data.shape[0]} have Lane ID as 7 or 8 which are the Ramps")


dataset before Row Removal(4566387, 33)
i-80: 153820 vehicles first and Last 5 seconds removed from Lane 1 due to lane changing
i-80: 157225 vehicles first and Last 5 seconds removed from Lane 2 due to lane changing
i-80: 163719 vehicles first and Last 5 seconds removed from Lane 3 due to lane changing
i-80: 830630 have multiple lengths and classes for same Vehicle ID
i-80: 940574 have total car following less than 30 seconds
i-80: 1002502 have time Headway less than 5 seconds
i-80: 2611277 have Lane ID as 7 or 8 which are the Ramps


In [25]:
remove = pd.concat(
    [bad_v_Class_length,time_headway_less_than5, total_duration_less_than_minute, both_lane_change_1, both_lane_change_2, both_lane_change_3, remove_ramp_data])

df = df.drop(labels=remove.index)
after = df.shape[0]
removed_row_count = before - after
print(f"{loc}: {removed_row_count} rows removed using above criterias")

MemoryError: Unable to allocate 89.4 MiB for an array with shape (2, 5859747) and data type float64

In [24]:
remove.drop_duplicates(inplace=True)

KeyboardInterrupt: 

In [26]:
both_lane_change_1[(both_lane_change_1['pair_Time_Duration'] < (both_lane_change_1['total_pair_duration'] - 5))& (both_lane_change_1['pair_Time_Duration'] > 5)]

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration


In [27]:
both_lane_change_2[(both_lane_change_2['pair_Time_Duration'] < (both_lane_change_2['total_pair_duration'] - 5))& (both_lane_change_2['pair_Time_Duration'] > 5)]

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration


In [28]:
both_lane_change_3[(both_lane_change_3['pair_Time_Duration'] < (both_lane_change_3['total_pair_duration'] - 5))& (both_lane_change_3['pair_Time_Duration'] > 5)]

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration


In [29]:
remove

MemoryError: Unable to allocate 134. MiB for an array with shape (3, 5859747) and data type int64

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration
0,36,4,1113433135300,8.268005,23.032822,7.55904,3,3.377184,0.000000,3,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,0.0,88.3
1,36,5,1113433135400,8.268005,23.337926,7.55904,3,3.377184,0.000000,3,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,0.1,88.3
2,36,6,1113433135500,8.268005,23.642422,7.55904,3,3.377184,0.000000,3,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,0.2,88.3
3,36,7,1113433135600,8.268310,23.794822,7.55904,3,3.377184,0.000000,3,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,0.3,88.3
4,36,8,1113433135700,8.420100,24.709222,7.55904,3,3.377184,0.000000,3,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,0.4,88.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4566382,2485,11687,1113438733600,16.769182,498.415361,4.20624,2,3.557016,-1.094232,5,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,5.8,6.2
4566383,2485,11688,1113438733700,16.771315,498.778073,4.20624,2,3.307080,-3.413760,5,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,5.9,6.2
4566384,2485,11689,1113438733800,16.774363,499.163340,4.20624,2,3.307080,0.000000,5,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,6.0,6.2
4566385,2485,11690,1113438733900,16.775278,499.315740,4.20624,2,3.307080,0.000000,5,...,0.0,True,0,0.0,0.0,False,9999.0,0.0,6.1,6.2


In [ ]:
remove

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration
83,50,49,1113433139800,9.273235,19.409359,4.66344,2,4.581144,0.000000,3,...,5.95884,True,36,5.419344,0.076200,True,43.784520,3.0,0.0,48.6
86,50,50,1113433139900,9.274454,19.866559,4.66344,2,4.581144,0.000000,3,...,5.95884,True,36,5.422392,0.036576,True,44.327064,3.0,0.1,48.6
88,50,51,1113433140000,9.275674,20.323454,4.66344,2,4.581144,0.000000,3,...,5.95884,True,36,5.422392,-0.033528,True,44.869913,3.0,0.2,48.6
93,50,52,1113433140100,9.275674,20.803514,4.66344,2,4.581144,0.000000,3,...,5.95884,True,36,5.419344,-0.073152,True,45.412152,3.0,0.3,48.6
94,50,53,1113433140200,9.275978,21.261019,4.66344,2,4.581144,0.000000,3,...,5.95884,True,36,5.413248,-0.045720,True,45.953477,3.0,0.4,48.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4566151,2485,11555,1113438720400,16.791127,480.571150,4.20624,2,3.450336,-3.413760,5,...,4.52628,True,2460,1.435608,-1.712976,True,493.227055,2.0,118.5,128.4
4566153,2485,11556,1113438720500,16.794175,480.889361,4.20624,2,2.941320,-3.413760,5,...,4.52628,True,2460,1.246632,-2.033016,True,493.363910,2.0,118.6,128.4
4566176,2485,11568,1113438721700,16.815816,482.972364,4.20624,2,2.194560,0.326136,5,...,4.52628,True,2460,0.396240,2.575560,True,493.834522,2.0,119.8,128.4
4566179,2485,11569,1113438721800,16.817645,483.201269,4.20624,2,2.157984,-0.944880,5,...,4.52628,True,2460,0.621792,2.609088,True,493.885118,2.0,119.9,128.4


In [ ]:
bad_v_Class_length

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration
0,36,4,1113433135300,8.268005,23.032822,7.55904,3,3.377184,0.0,3,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.0,88.3
1,36,5,1113433135400,8.268005,23.337926,7.55904,3,3.377184,0.0,3,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.1,88.3
2,36,6,1113433135500,8.268005,23.642422,7.55904,3,3.377184,0.0,3,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.2,88.3
3,36,7,1113433135600,8.268310,23.794822,7.55904,3,3.377184,0.0,3,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.3,88.3
4,36,8,1113433135700,8.420100,24.709222,7.55904,3,3.377184,0.0,3,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.4,88.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4566024,2460,11502,1113438715100,16.737482,478.060207,4.66344,2,3.319272,0.0,5,...,7.02564,True,2445,1.030224,0.624840,True,499.102380,3.0,4.7,5.1
4566026,2460,11503,1113438715200,16.741750,478.392134,4.66344,2,3.319272,0.0,5,...,7.02564,True,2445,1.136904,1.420368,True,499.203574,3.0,4.8,5.1
4566030,2460,11504,1113438715300,16.746322,478.724062,4.66344,2,3.319272,0.0,5,...,7.02564,True,2445,1.136904,0.000000,True,499.311778,3.0,4.9,5.1
4566034,2460,11505,1113438715400,16.750589,479.055684,4.66344,2,3.319272,0.0,5,...,7.02564,True,2445,1.136904,0.000000,True,499.464482,3.0,5.0,5.1


In [ ]:
filtered_ngsim_I.columns

Index(['Vehicle_ID', 'Frame_ID', 'Global_Time', 'Local_X', 'Local_Y',
       'v_length', 'v_Class', 'v_Vel', 'v_Acc', 'Lane_ID', 'Preceding',
       'Following', 'Space_Headway', 'Time_Headway', 'Location',
       'Preceding_Vehicle_Class', 'Rear_to_Front_Space_Headway',
       'Front_To_Rear_Time_Headway', 'Velocity Difference_Following-Preceding',
       'Acceleration Difference_Following-Preceding', 'L-F_Pair',
       'v_Class_Name', 'Vehicle_combination', 'preceding_vehicle_length',
       'lane_changes', 'Prec_Vehicle_ID', 'preceding_Vehicle_Velocity',
       'preceding_Vehicle_Acceleration', 'preceding_car_lane_changes',
       'preceding_Local_Y', 'preceding_v_Class', 'pair_Time_Duration',
       'total_pair_duration'],
      dtype='object')

In [ ]:
filtered_ngsim_I[filtered_ngsim_I['total_pair_duration']<30]['Vehicle_ID'].count()
filtered_ngsim_U[filtered_ngsim_U['total_pair_duration']<30]['Vehicle_ID'].count()

780619

In [ ]:
#dsasas

NameError: name 'dsasas' is not defined

In [ ]:
filtered_ngsim_U.drop(columns=['pair_Time_Duration'],axis=1,inplace=True)
filtered_ngsim_I.drop(columns=['pair_Time_Duration'],axis=1,inplace=True)
filtered_ngsim_U=transform.remap_pair_time(filtered_ngsim_U)
filtered_ngsim_I=transform.remap_pair_time(filtered_ngsim_I)

In [ ]:
filtered_ngsim_I['diference'] = filtered_ngsim_I['total_pair_duration'] - filtered_ngsim_I['total_pair_dur'] 

In [ ]:
filtered_ngsim_I[filtered_ngsim_I['diference'] >10]

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,total_pair_duration,pair_Time_Duration,total_pair_dur,diference
45,11,57,1113433140600,5.463235,19.419113,5.27304,2,3.819144,0.000000,2,...,1,3.959352,0.079248,False,32.841895,2.0,83.8,0.0,69.9,13.9
47,11,58,1113433140700,5.433670,19.800113,5.27304,2,3.819144,0.000000,2,...,1,3.956304,-0.100584,False,33.239659,2.0,83.8,0.1,69.9,13.9
49,11,59,1113433140800,5.404104,20.181113,5.27304,2,3.819144,0.000000,2,...,1,3.861816,-1.459992,False,33.643519,2.0,83.8,0.2,69.9,13.9
52,11,60,1113433140900,5.374234,20.575219,5.27304,2,3.819144,0.000000,2,...,1,3.596640,-3.413760,False,34.033054,2.0,83.8,0.3,69.9,13.9
53,11,61,1113433141000,5.344058,20.956524,5.27304,2,3.819144,0.000000,2,...,1,3.142488,-3.413760,False,34.365286,2.0,83.8,0.4,69.9,13.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992599,2480,11235,1113438688400,9.011412,473.917670,4.35864,2,7.040880,0.000000,3,...,2483,2.505456,0.606552,True,491.075777,2.0,80.7,32.2,32.6,48.1
992600,2480,11236,1113438688500,9.031834,474.621758,4.35864,2,7.040880,0.000000,3,...,2483,2.545080,0.018288,True,491.333333,2.0,80.7,32.3,32.6,48.1
992601,2480,11237,1113438688600,9.052255,475.325846,4.35864,2,7.040880,0.000000,3,...,2483,2.551176,0.000000,True,491.588755,2.0,80.7,32.4,32.6,48.1
992602,2480,11238,1113438688700,9.072372,476.029020,4.35864,2,7.046976,0.124968,3,...,2483,2.554224,0.118872,True,491.842958,2.0,80.7,32.5,32.6,48.1


In [ ]:
filtered_ngsim_I[filtered_ngsim_I['total_pair_duration']<30]['Vehicle_ID'].count()
filtered_ngsim_I[filtered_ngsim_I['total_pair_dur']<30]['Vehicle_ID'].count()


132948

In [ ]:
asasa


1. Velocity difference of the Lead minus Preceding Vehicle.
2. Acceleration difference of the Lead minus Preceding Vehicle.
3. Find the Pair details of the Lead_following 
4. Find the cumulative time the Pair runs within the highway as a pair. 

In [ ]:
filtered_ngsim= fileProcessing.merge_files(filtered_ngsim_U , filtered_ngsim_I)
filtered_ngsim=cleanup.remove_dup_pairs(filtered_ngsim)
x_hv_car=x[x.index.get_level_values('Vehicle_combination').isin(['Heavy Vehicle-Car'])]
hv_car_pair= x_hv_car[x_hv_car>60].tail(3).index.get_level_values('L-F_Pair').values
hv_car_pair
x_car_car=x[x.index.get_level_values('Vehicle_combination').isin(['Car-Car'])]
Car_car_pair= x_car_car[x_car_car>60].tail(3).index.get_level_values('L-F_Pair').values
Car_car_pair
x_car_hv=x[x.index.get_level_values('Vehicle_combination').isin(['Car-Heavy Vehicle'])]
car_hv_pair= x_car_hv[x_car_hv>60].tail(3).index.get_level_values('L-F_Pair').values
car_hv_pair


 Merged Record Count:2634976, df1:1643335, df2:991641
0 rows removed for Dups. Count Before removal: 2634976, Count After Removal:2634976


True

In [ ]:
fileProcessing.export_file(filtered_ngsim,'Cleaned_NGSIM_Data')
filtered_ngsim.columns

Index(['Vehicle_ID', 'Frame_ID', 'Global_Time', 'Local_X', 'Local_Y',
       'v_length', 'v_Class', 'v_Vel', 'v_Acc', 'Lane_ID', 'Preceding',
       'Following', 'Space_Headway', 'Time_Headway', 'Location',
       'Preceding_Vehicle_Class', 'Rear_to_Front_Space_Headway',
       'Front_To_Rear_Time_Headway', 'Velocity Difference_Following-Preceding',
       'Acceleration Difference_Following-Preceding', 'L-F_Pair',
       'v_Class_Name', 'Vehicle_combination', 'preceding_vehicle_length',
       'lane_changes', 'Prec_Vehicle_ID', 'preceding_Vehicle_Velocity',
       'preceding_Vehicle_Acceleration', 'preceding_car_lane_changes',
       'preceding_Local_Y', 'preceding_v_Class', 'pair_Time_Duration',
       'total_pair_duration'],
      dtype='object')

In [ ]:
print(f"{filtered_ngsim[(filtered_ngsim['preceding_car_lane_changes'] == True) ]['Vehicle_ID'].unique().size} cars Change lanes")
print(f"{filtered_ngsim[(filtered_ngsim['preceding_car_lane_changes'] == False) ]['Vehicle_ID'].unique().size} cars DonT")

2171 cars Change lanes
833 cars DonT


In [ ]:
#filtered_ngsim[((filtered_ngsim['Vehicle_ID'] <= 100 )  ) & ((filtered_ngsim['Preceding'] <= 100 )  )].to_csv('classVERIFYFILE.CSV', index = False)

In [ ]:
filtered_ngsim

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration
0,2,13,1118846980200,5.019142,10.784129,4.41960,2,12.192000,0.000000,2,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.0,88.1
1,2,14,1118846980300,5.013046,12.003329,4.41960,2,12.192000,0.000000,2,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.1,88.1
2,2,15,1118846980400,5.006645,13.222529,4.41960,2,12.192000,0.000000,2,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.2,88.1
3,2,16,1118846980500,5.000244,14.441424,4.41960,2,12.192000,0.000000,2,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.3,88.1
4,2,17,1118846980600,4.994148,15.660929,4.41960,2,12.192000,0.000000,2,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.4,88.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991636,2480,11235,1113438688400,9.011412,473.917670,4.35864,2,7.040880,0.000000,3,...,4.61264,True,2483,2.505456,0.606552,True,491.075777,2.0,32.2,32.6
991637,2480,11236,1113438688500,9.031834,474.621758,4.35864,2,7.040880,0.000000,3,...,4.61264,True,2483,2.545080,0.018288,True,491.333333,2.0,32.3,32.6
991638,2480,11237,1113438688600,9.052255,475.325846,4.35864,2,7.040880,0.000000,3,...,4.61264,True,2483,2.551176,0.000000,True,491.588755,2.0,32.4,32.6
991639,2480,11238,1113438688700,9.072372,476.029020,4.35864,2,7.046976,0.124968,3,...,4.61264,True,2483,2.554224,0.118872,True,491.842958,2.0,32.5,32.6


In [ ]:
filtered_ngsim['L-F_Pair'].unique().shape

(4817,)

In [ ]:
filtered_ngsim[((filtered_ngsim['Space_Headway'] == 0 ) & (filtered_ngsim['Preceding'] > 0 )) ]


,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration
456259,2964,8731,1118847852000,1.483766,94.405399,5.02920,2,2.618232,-3.413760,1,...,4.72440,False,2956,1.411224,1.298448,False,94.406923,2.0,4.7,57.5
570284,2491,7916,1113437558500,5.230673,182.539843,4.66344,2,3.453384,0.036576,2,...,4.51104,True,2486,3.200400,0.000000,False,182.540148,2.0,29.3,89.6
833036,1748,6068,1113438171700,5.372100,22.727107,5.42544,2,7.504176,0.000000,2,...,4.35864,True,1734,7.620000,0.000000,True,22.727412,2.0,0.2,29.7


In [ ]:
filtered_ngsim[filtered_ngsim['total_pair_duration']<30]

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration
2148,10,258,1118847004700,1.903476,303.685651,5.18160,2,14.599920,3.413760,1,...,5.23240,True,2,17.141952,-2.831592,True,337.310578,2.0,0.0,19.1
2174,10,259,1118847004800,1.886102,305.166065,5.18160,2,14.910816,2.880360,1,...,5.23240,True,2,16.919448,-1.789176,True,339.004046,2.0,0.1,19.1
2199,10,260,1118847004900,1.890370,306.680311,5.18160,2,15.127224,1.533144,1,...,5.23240,True,2,16.794480,-0.627888,True,340.679227,2.0,0.2,19.1
2208,10,261,1118847005000,1.896161,308.208883,5.18160,2,15.230856,0.371856,1,...,5.23240,True,2,16.754856,0.021336,True,342.350750,2.0,0.3,19.1
2231,10,262,1118847005100,1.898294,309.737760,5.18160,2,15.252192,-0.109728,1,...,5.23240,True,2,16.757904,0.076200,True,344.025626,2.0,0.4,19.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991046,2435,10954,1113438660300,9.559138,482.097283,4.66344,2,2.968752,-3.407664,3,...,3.38328,True,2437,0.612648,0.920496,True,489.121094,2.0,27.9,28.3
991056,2435,10955,1113438660400,9.564319,482.370689,4.66344,2,2.566416,-3.413760,3,...,3.38328,True,2437,0.643128,-0.441960,True,489.195770,2.0,28.0,28.3
991062,2435,10956,1113438660500,9.571939,482.602946,4.66344,2,2.075688,-3.413760,3,...,3.38328,True,2437,0.621792,-0.445008,True,489.259169,2.0,28.1,28.3
991065,2435,10957,1113438660600,9.582302,482.792532,4.66344,2,1.658112,-3.413760,3,...,3.38328,True,2437,0.649224,0.877824,True,489.311290,2.0,28.2,28.3
